In [ ]:
import numpy as np
import pandas as pd
import shap
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from joblib import load
import matplotlib.pyplot as pl
import pickle



### Data preprocessing

In [ ]:
data = pd.read_csv('dataset_postprocessed.csv')
data.dropna()
data = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
y_train_data = data["class"] 
x_train_data = data.drop(["class", "latitude", "longitude", "spring_product_name", "autumn_product_name", "summer_product_name"], axis=1)
pc_columns = x_train_data.columns

reduced_x_train_data = data[pc_columns]
reduced_x_train_data.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reduced_x_train_data, y_train_data, test_size=0.15, random_state=0,)
labels=y_train_data.unique()

In [ ]:
model = load('model.joblib')
y_true = model.predict(X_test)

In [ ]:
pred = pd.DataFrame(y_true).reset_index(drop=True, inplace=False)
real = y_test.reset_index(drop=True, inplace=False)
test = pd.DataFrame(X_test).reset_index(drop=True, inplace=False)
train = pd.DataFrame(X_train).reset_index(drop=True, inplace=False)

# Explainability with SHAP

All dataset is found in *reduced_x_train_data*. The name has not been changed to not modify the previus code.

In [ ]:
explainer = shap.TreeExplainer(model)

In [ ]:
choosen_instance = reduced_x_train_data
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()

In [ ]:
with open("matriz.pckl","wb") as f:
    pickle.dump(shap_values, f)

In [ ]:
#to import the matrix again
shap_values = pickle.load(open('matriz.pckl', 'rb'))

## Using all the features


In [ ]:
labels

In [ ]:
pretty_labels= ['Built Up', 'Water', 'Open Forest', 'Closed Forest', 'Bare Soil',
       'Cropland', 'Herbaceous Vegetation', 'Wetland', 'Shrubland']

In [ ]:
num_features = len(list(reduced_x_train_data.columns.values))

In [ ]:
def calculateColors(i):
    #           Water  ClosedForest  cropland openForest bareSoil HerbaceousVeg  BuiltUp    wetland     shrubland
    colors = ["#fa0000", "#0032c8", "#648c00", "#007800", "#b4b4b4", "#f096ff", "#ffff4c", "#0096a0", "#ffbb22"]
    return colors[i]

In [ ]:
shap.summary_plot(shap_values, list(reduced_x_train_data.columns.values),  max_display =num_features, class_names=pretty_labels,  color= calculateColors, class_inds="original", show=False)
pl.savefig("all_features_DEF.png")


## Dividing the features in 3 

In this part we're going to divive the dataset in 3 halves. For this porpose the *train dataset* and the *shap_values* (the dimennsion corresponding to features) will be reduced to the halved 

In [ ]:
half = int(np.round(num_features/3))


In [ ]:
feature_order = np.argsort(np.sum(np.mean(np.abs(shap_values), axis=1), axis=0))
feature_order[half*2:]

#### Crop the shap_values

In [ ]:
shap_values_array = np.array(shap_values)
cropped_first_shap_values = list(shap_values_array[:,:,feature_order[:half]])
cropped_second_shap_values = list(shap_values_array[:,:,feature_order[half: half*2]])
cropped_thrid_shap_values = list(shap_values_array[:,:,feature_order[half*2:]])

# alternatively, you can do directly -- > list_first_shap_values = [shap_values[i][:,:36] for i in range(len(shap_values))] 

#### Crop the train dataset

In [ ]:
first_half_train = reduced_x_train_data.iloc[:,feature_order[:half]]
second_half_train = reduced_x_train_data.iloc[:,feature_order[half: half*2]]
thrid_half_train = reduced_x_train_data.iloc[:,feature_order[half*2:]]

### Plot for the first half

For both plots, it's important to specicy **class_inds="original"** to set the labels in the correct order

In [ ]:
shap.summary_plot(cropped_first_shap_values,  first_half_train.columns.values, max_display = half, class_names=pretty_labels,  color= calculateColors, class_inds="original", show=False )
pl.xlim([0, 0.12])
pl.legend(loc='lower right')
pl.savefig("first_half_DEF.png")


### Plot for the second half

In [ ]:
shap.summary_plot(cropped_second_shap_values,  second_half_train.columns.values, max_display = half, class_names=pretty_labels,  color= calculateColors, show=False )
pl.xlim([0, 0.12])
pl.legend(loc='lower right')
pl.savefig("second_half_DEF.png")


### Plot for the second half

In [ ]:
shap.summary_plot(cropped_thrid_shap_values, thrid_half_train.columns.values, max_display = half, class_names=pretty_labels,  color= calculateColors, show=False )
pl.xlim([0, 0.12])
pl.legend(loc='lower right')
pl.savefig("third_half_DEF.png")

## Doing transpose - Discarded
Initially the form of **shap_values** is (classes, samples, features). You can do a transpose to change the dimensions of the matrix to get (features, samples, classes), however this does not make any sense because you cannot change the input to the model. (The model cannot have 9 classes as input and predict a feature).

In [ ]:
np.shape(shap_values)

In [ ]:
shap_values_transpose = np.transpose(shap_values, (2, 1, 0))

In [ ]:
np.shape(shap_values_transpose)

In [ ]:
np.shape(reduced_x_train_data.index.values)

In [ ]:
reduced_x_train_data.columns.values

In [ ]:
np.array(pretty_labels)

In [ ]:

#shap.summary_plot(cropped_first_shap_values,  first_half_train.columns.values, max_display = half, class_names=pretty_labels,  color= calculateColors, class_inds="original", show=False )
shap.summary_plot(list(shap_values_transpose), np.array(pretty_labels), class_names=reduced_x_train_data.columns.values,  max_display = 9,class_inds="original",  show=False )
pl.legend(loc=(1.04, 0))
pl.savefig("transpose_plot_DEF1.png", bbox_inches = 'tight')
pl.tight_layout()


### Split by class

In [ ]:
def data_train(data, pc_columns):
    y_train_data = data["class"] 
    x_train_data = data.drop(["class", "latitude", "longitude", "spring_product_name", "autumn_product_name", "summer_product_name"], axis=1)

    reduced_x_train_data = data[pc_columns]
    #reduced_x_train_data.to_csv(f'{label}.csv')

    X_train, X_test, y_train, y_test = train_test_split(reduced_x_train_data, y_train_data, test_size=0.50, random_state=0,)

    y_true = model.predict(X_test)

    X_train = pd.DataFrame(X_train).reset_index(drop=True, inplace=False)
    X_test = pd.DataFrame(X_test).reset_index(drop=True, inplace=False)
    y_test = pd.DataFrame(y_test).reset_index(drop=True, inplace=False)
    y_train = pd.DataFrame(y_train).reset_index(drop=True, inplace=False)
    y_true = pd.DataFrame(y_true).reset_index(drop=True, inplace=False)
    

    return X_train, X_test, y_train, y_test, y_true


In [ ]:
labels = ['closedForest']
print(labels)

In [ ]:
for label in labels:
    b_aux = data['class'] == label
    b = data[b_aux]
    X_train, X_test, y_train, y_test, y_true = data_train(b, pc_columns)
    explainer = shap.TreeExplainer(model)
    choosen_instance = X_test.loc[0:10]
    shap_values = explainer.shap_values(choosen_instance)
    shap.initjs()
    shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)


In [ ]:
explainer = shap.TreeExplainer(model)
choosen_instance = X_test.loc[0:3]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)